In [22]:
import os
import glob
import pandas as pd
from pathlib import Path
from scipy import stats

In [54]:
def run_cv_analysis(base_path):
    paths = list(filter(lambda x: x is not None, [path if path.is_dir() else None for path in\
         Path(base_path).iterdir()]))

    file = pd.read_csv(paths[0] / 'pred.csv').file

    cols = ["file"]
    cols.extend([f"prognosis_{ind}"for ind in range(len(list(paths)))])
    preds = pd.DataFrame(columns=cols)

    preds["file"] = file

    for ind, path_dir in enumerate(paths):
        pred = pd.read_csv(path_dir / 'pred.csv')
        preds[f"prognosis_{ind}"] = pred.prognosis
        
    modes = stats.mode(preds.values[:, 1:], axis=1)[0]
    preds["mode_prognosis"] = modes
    
    preds.to_csv(os.path.join(base_path, 'mode_preds.csv'),
             index=False, columns=["file", "mode_prognosis"])

    preds['mode_prognosis'].replace('SEVERE', 1, inplace=True)
    preds['mode_prognosis'].replace('MILD', 0, inplace=True)

    for ind, path_dir in enumerate(paths):
        preds[f"prognosis_{ind}"].replace('SEVERE', 1, inplace=True)
        preds[f"prognosis_{ind}"].replace('MILD', 0, inplace=True)
        
    test_raw_df = pd.read_excel('/mnt/ncshare/ai4covid_hackathon/raw_data/completeTestClinData.xls')
    test_raw_df = test_raw_df[ ['ImageFile', 'Prognosis'] ]
    test_raw_df.Prognosis.replace('SEVERE', 1, inplace=True)
    test_raw_df.Prognosis.replace('MILD', 0, inplace=True)
    test_raw_df.head()
    
    scores = []

    for ind, path_dir in enumerate(paths):
        scores.append(balanced_accuracy_score(test_raw_df.Prognosis, preds[f"prognosis_{ind}"]))

    print('Balanced accuracy score (CV):\t', np.mean(scores), '+/-', np.std(scores))
    print('Balanced accuracy score (mode):\t', balanced_accuracy_score(test_raw_df.Prognosis, preds["mode_prognosis"]))

In [33]:
paths = list(filter(lambda x: x is not None, [path if path.is_dir() else None for path in\
         Path('/home/qbeer/GitHub/ai4covid/data/raw_output/checkpoints/CROSS_VAL_paper_v1_w_ES/').iterdir()]))

file = pd.read_csv(paths[0] / 'pred.csv').file

cols = ["file"]
cols.extend([f"prognosis_{ind}"for ind in range(len(list(paths)))])
preds = pd.DataFrame(columns=cols)

preds["file"] = file

for ind, path_dir in enumerate(paths):
    pred = pd.read_csv(path_dir / 'pred.csv')
    preds[f"prognosis_{ind}"] = pred.prognosis

In [34]:
preds.head()

,file,prognosis_0,prognosis_1,prognosis_2,prognosis_3,prognosis_4
0,P_3_29.png,SEVERE,SEVERE,SEVERE,SEVERE,MILD
1,P_3_415.png,MILD,MILD,MILD,MILD,MILD
2,P_3_163.png,MILD,MILD,MILD,MILD,MILD
3,P_3_348.png,SEVERE,MILD,SEVERE,SEVERE,MILD
4,P_3_263.png,SEVERE,SEVERE,SEVERE,SEVERE,MILD


In [35]:
preds.describe()

,file,prognosis_0,prognosis_1,prognosis_2,prognosis_3,prognosis_4
count,486,486,486,486,486,486
unique,486,2,2,2,2,2
top,P_3_330.png,SEVERE,MILD,MILD,MILD,MILD
freq,1,289,250,299,271,250


In [36]:
modes = stats.mode(preds.values[:, 1:], axis=1)[0]
preds["mode_prognosis"] = modes

In [37]:
preds

,file,prognosis_0,prognosis_1,prognosis_2,prognosis_3,prognosis_4,mode_prognosis
0,P_3_29.png,SEVERE,SEVERE,SEVERE,SEVERE,MILD,SEVERE
1,P_3_415.png,MILD,MILD,MILD,MILD,MILD,MILD
2,P_3_163.png,MILD,MILD,MILD,MILD,MILD,MILD
3,P_3_348.png,SEVERE,MILD,SEVERE,SEVERE,MILD,SEVERE
4,P_3_263.png,SEVERE,SEVERE,SEVERE,SEVERE,MILD,SEVERE
...,...,...,...,...,...,...,...
481,P_3_432.png,MILD,MILD,MILD,MILD,MILD,MILD
482,P_3_105.png,SEVERE,SEVERE,SEVERE,SEVERE,SEVERE,SEVERE
483,P_3_233.png,MILD,MILD,MILD,MILD,MILD,MILD
484,P_3_414.png,SEVERE,MILD,SEVERE,MILD,SEVERE,SEVERE


In [46]:
preds.to_csv('/home/qbeer/GitHub/ai4covid/data/raw_output/checkpoints/CROSS_VAL_paper_v1_w_ES/mode_preds.csv',
             index=False, columns=["file", "mode_prognosis"])

preds['mode_prognosis'].replace('SEVERE', 1, inplace=True)
preds['mode_prognosis'].replace('MILD', 0, inplace=True)

for ind, path_dir in enumerate(paths):
    preds[f"prognosis_{ind}"].replace('SEVERE', 1, inplace=True)
    preds[f"prognosis_{ind}"].replace('MILD', 0, inplace=True)

In [47]:
from sklearn.metrics import balanced_accuracy_score

In [48]:
test_raw_df = pd.read_excel('/mnt/ncshare/ai4covid_hackathon/raw_data/completeTestClinData.xls')
test_raw_df = test_raw_df[ ['ImageFile', 'Prognosis'] ]
test_raw_df.Prognosis.replace('SEVERE', 1, inplace=True)
test_raw_df.Prognosis.replace('MILD', 0, inplace=True)
test_raw_df.head()

,ImageFile,Prognosis
0,P_3_29.png,0
1,P_3_415.png,0
2,P_3_163.png,0
3,P_3_348.png,1
4,P_3_263.png,1


In [52]:
import numpy as np

scores = []

for ind, path_dir in enumerate(paths):
    scores.append(balanced_accuracy_score(test_raw_df.Prognosis, preds[f"prognosis_{ind}"]))
    
print(np.mean(scores), '+/-', np.std(scores))

0.7243464052287582 +/- 0.017661577179603757


In [53]:
balanced_accuracy_score(test_raw_df.Prognosis, preds["mode_prognosis"])

0.7338235294117648

In [55]:
run_cv_analysis('/mnt/ncshare/ai4covid_hackathon/raw_output/checkpoints/ENSEMBLE/')

Balanced accuracy score (CV):	 0.7078022875816994 +/- 0.021456113431867808
Balanced accuracy score (mode):	 0.7496732026143791


In [ ]:
## Same for image only
